In [ ]:
import numpy as np
from pathlib import Path

from pymor.basic import *
from pymor.core.pickle import load

from RBInvParam.problems.problems import build_InstationaryModelIP

set_log_levels({
    'pymor' : 'WARN'
})

set_defaults({})


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "cm",
    "font.size": 10,
    'text.latex.preamble': r'\usepackage{amsfonts} \usepackage{accents} \usepackage{mathrsfs}',
    'figure.dpi': 400
})

In [ ]:
#WORK_DIR = Path('/run/media/benedikt/38E84473425D444F/Dokumente/Studium/Mathematik/PhD/parabolische_inverse_probleme')
WORK_DIR = Path('/home/benedikt/Dokumente/parabolische_inverse_probleme')
data_dir_path = WORK_DIR / Path('experiments/test_POD_tol')
#data_dir_path = WORK_DIR / Path('experiments/enforce_adm_region_corrected_suff_cond')

FOM_data_path = Path('/home/benedikt/Dokumente/parabolische_inverse_probleme/experiments/enforce_adm_region_corrected_suff_cond') / 'reaction_FOM_300_non_time_dep'
#TR_data_path = data_dir_path / 'diffusion_TR_300_non_time_dep'
TR_data_path = data_dir_path / 'reaction_TR_non_time_dep_1e-3'

setup_path = FOM_data_path / 'setup.pkl'
optimizer_parameter_path = FOM_data_path / 'optimizer_parameter.pkl'


with open(setup_path, 'rb') as file:
    setup = load(file)

with open(optimizer_parameter_path, 'rb') as file:
    optimizer_parameter = load(file)

if not 'FOM' in locals():
    FOM, grid_data, assembled_products = build_InstationaryModelIP(setup=setup)

    

data = {}

if FOM_data_path:
    with open(FOM_data_path / 'FOM_IRGNM_final.pkl', 'rb') as file:
        FOM_data = load(file)
    data['FOM'] = FOM_data

if TR_data_path:
    with open(TR_data_path / 'TR_IRGNM_final.pkl', 'rb') as file:
        TR_data = load(file)
    data['TR'] = TR_data
    


In [ ]:
from typing import Dict, Tuple
from RBInvParam.products import BochnerProductOperator

def calc_rel_error(data: Dict) -> Tuple[float, float]:

    
    q_FOM = data['FOM']['q'][-1]
    q_TR = data['TR']['q'][-1]

    if setup['model_parameter']['q_time_dep']:
        nt = setup['dims']['nt']
    else:
        nt = 1
    
    bochner_L2 = BochnerProductOperator(
        product=NumpyMatrixOperator(
            matrix = assembled_products['l2'],
            source_id = FOM.Q.id,
            range_id = FOM.Q.id
        ),
        delta_t=setup['model_parameter']['delta_t'],
        space = FOM.Q,
        nt = nt
    )
    
    L2_rel_error = np.sqrt(bochner_L2.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_L2.apply2(q_FOM, q_FOM)[0,0])
    print("L2 rel error:")
    print(f"{L2_rel_error:.2e}")
    
    
    bochner_H1 = BochnerProductOperator(
        product=NumpyMatrixOperator(
            matrix = assembled_products['h1'],
            source_id = FOM.Q.id,
            range_id = FOM.Q.id
        ),
        delta_t=setup['model_parameter']['delta_t'],
        space = FOM.Q,
        nt = nt
    )
    H1_rel_error = np.sqrt(bochner_H1.apply2(q_FOM-q_TR, q_FOM-q_TR)[0,0]) / np.sqrt(bochner_H1.apply2(q_FOM, q_FOM)[0,0])
    print("H1 rel error:")
    print(f"{H1_rel_error:.2e}")

    return L2_rel_error, H1_rel_error



In [ ]:
data['TR']['FOM_num_calls']

In [ ]:
import pandas as pd

columns = ['Algo.', 
           r'$\eps_\text{POD}$',
           r'$L^2$-rel. err.',
           r'$H^1$-rel. err.',
           'time [s]', 
           'speed up', 
           'FOM solves', 
           r'$n_Q$', 
           r'$n_V$', 
           'o. iter']

rows = []
algo = 'FOM'
FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
             data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']
row = [algo,
       '--',
       '--',
       '--',
       int(data[algo]['total_runtime'][-1]),
       '--',
       FOM_solves,
       '--',
       '--',
       len(data[algo]['J'])-1]
rows.append(row)

algo = 'TR'
FOM_solves = data[algo]['FOM_num_calls']['solve_state'] + data[algo]['FOM_num_calls']['solve_adjoint'] + \
             data[algo]['FOM_num_calls']['solve_linearized_state'] + data[algo]['FOM_num_calls']['solve_linearized_adjoint']

L2_rel_error, H1_rel_error = calc_rel_error(data)



row = [algo,
       r'$10^{-??}$',
       L2_rel_error,
       H1_rel_error,
       int(data[algo]['total_runtime'][-1]),
       '--',
       int(FOM_solves),
       data[algo]['dim_Q_r'][-1],
       data[algo]['dim_V_r'][-1],
       len(data[algo]['J'])-1]
rows.append(row)


df = pd.DataFrame.from_records(rows, columns=columns)
speed_up = df['time [s]'][0] / df['time [s]'][1]
df['speed up'][1] = f'{speed_up:.2f}'

In [ ]:
print(df.to_latex(index=False, float_format="{:.2e}".format, column_format='l|cccccc'))